# Food Demand Model

In [48]:
# Import modules 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle as pk
import seaborn as sns

#from imblearn.over_sampling import SMOTE                   #Import when unbalanced data
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [50]:
# import food dataset
train = pd.read_csv("Food_train.csv")
test = pd.read_csv("FoodTest.csv")

In [51]:
train.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
0,1379560,1,55,1885,136.83,152.29,0,0,177
1,1466964,1,55,1993,136.83,135.83,0,0,270
2,1346989,1,55,2539,134.86,135.86,0,0,189
3,1338232,1,55,2139,339.50,437.53,0,0,54
4,1448490,1,55,2631,243.50,242.50,0,0,40


In [52]:
test.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured
0,1028232,146,55,1885,158.11,159.11,0,0
1,1127204,146,55,1993,160.11,159.11,0,0
2,1212707,146,55,2539,157.14,159.14,0,0
3,1082698,146,55,2631,162.02,162.02,0,0
4,1400926,146,55,1248,163.93,163.93,0,0


In [53]:
train.shape

(456548, 9)

In [54]:
test.shape

(32573, 8)

In [55]:
train.isnull().sum()

id                       0
week                     0
center_id                0
meal_id                  0
checkout_price           0
base_price               0
emailer_for_promotion    0
homepage_featured        0
num_orders               0
dtype: int64

In [56]:
#import centre dataset
centreDF = pd.read_csv("center.csv")
centreDF.head()

,center_id,city_code,region_code,center_type,op_area
0,11,679,56,TYPE_A,3.7
1,13,590,56,TYPE_B,6.7
2,124,590,56,TYPE_C,4.0
3,66,648,34,TYPE_A,4.1
4,94,632,34,TYPE_C,3.6


In [57]:
#Prepare main dataset DF by mergeing centre dataset and Meal dataset
train = pd.merge(train,centreDF, on=['center_id'])
test = pd.merge(test,centreDF, on=['center_id'])

In [58]:
#import centre dataset
mealDF = pd.read_csv("meal.csv")
mealDF.head()

,meal_id,category,cuisine
0,1885,Beverages,Thai
1,1993,Beverages,Thai
2,2539,Beverages,Thai
3,1248,Beverages,Indian
4,2631,Beverages,Indian


In [59]:
#Prepare main dataset DF by mergeing centre dataset and Meal dataset
train = pd.merge(train,mealDF, on=['meal_id'])
test = pd.merge(test,mealDF, on=['meal_id'])

In [60]:
train.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,city_code,region_code,center_type,op_area,category,cuisine
0,1379560,1,55,1885,136.83,152.29,0,0,177,647,56,TYPE_C,2.0,Beverages,Thai
1,1018704,2,55,1885,135.83,152.29,0,0,323,647,56,TYPE_C,2.0,Beverages,Thai
2,1196273,3,55,1885,132.92,133.92,0,0,96,647,56,TYPE_C,2.0,Beverages,Thai
3,1116527,4,55,1885,135.86,134.86,0,0,163,647,56,TYPE_C,2.0,Beverages,Thai
4,1343872,5,55,1885,146.50,147.50,0,0,215,647,56,TYPE_C,2.0,Beverages,Thai


In [61]:
test.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,city_code,region_code,center_type,op_area,category,cuisine
0,1028232,146,55,1885,158.11,159.11,0,0,647,56,TYPE_C,2.0,Beverages,Thai
1,1262649,147,55,1885,159.11,159.11,0,0,647,56,TYPE_C,2.0,Beverages,Thai
2,1453211,149,55,1885,157.14,158.14,0,0,647,56,TYPE_C,2.0,Beverages,Thai
3,1262599,150,55,1885,159.14,157.14,0,0,647,56,TYPE_C,2.0,Beverages,Thai
4,1495848,151,55,1885,160.11,159.11,0,0,647,56,TYPE_C,2.0,Beverages,Thai


In [62]:
train.describe()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,city_code,region_code,op_area
count,4.565480e+05,456548.000000,456548.000000,456548.000000,456548.000000,456548.000000,456548.000000,456548.00000,456548.000000,456548.000000,456548.000000,456548.000000
mean,1.250096e+06,74.768771,82.105796,2024.337458,332.238933,354.156627,0.081152,0.10920,261.872760,601.553399,56.614566,4.083590
std,1.443548e+05,41.524956,45.975046,547.420920,152.939723,160.715914,0.273069,0.31189,395.922798,66.195914,17.641306,1.091686
min,1.000000e+06,1.000000,10.000000,1062.000000,2.970000,55.350000,0.000000,0.00000,13.000000,456.000000,23.000000,0.900000
25%,1.124999e+06,39.000000,43.000000,1558.000000,228.950000,243.500000,0.000000,0.00000,54.000000,553.000000,34.000000,3.600000
50%,1.250184e+06,76.000000,76.000000,1993.000000,296.820000,310.460000,0.000000,0.00000,136.000000,596.000000,56.000000,4.000000
75%,1.375140e+06,111.000000,110.000000,2539.000000,445.230000,458.870000,0.000000,0.00000,324.000000,651.000000,77.000000,4.500000
max,1.499999e+06,145.000000,186.000000,2956.000000,866.270000,866.270000,1.000000,1.00000,24299.000000,713.000000,93.000000,7.000000


In [63]:
train.isnull().sum()

id                       0
week                     0
center_id                0
meal_id                  0
checkout_price           0
base_price               0
emailer_for_promotion    0
homepage_featured        0
num_orders               0
city_code                0
region_code              0
center_type              0
op_area                  0
category                 0
cuisine                  0
dtype: int64

In [64]:
test.isnull().sum()

id                       0
week                     0
center_id                0
meal_id                  0
checkout_price           0
base_price               0
emailer_for_promotion    0
homepage_featured        0
city_code                0
region_code              0
center_type              0
op_area                  0
category                 0
cuisine                  0
dtype: int64

In [65]:
train.columns

Index(['id', 'week', 'center_id', 'meal_id', 'checkout_price', 'base_price',
       'emailer_for_promotion', 'homepage_featured', 'num_orders', 'city_code',
       'region_code', 'center_type', 'op_area', 'category', 'cuisine'],
      dtype='object')

In [66]:
for col in train.columns:
    print("Unique values in ",col," : ",train[col].nunique())

Unique values in  id  :  456548
Unique values in  week  :  145
Unique values in  center_id  :  77
Unique values in  meal_id  :  51
Unique values in  checkout_price  :  1992
Unique values in  base_price  :  1907
Unique values in  emailer_for_promotion  :  2
Unique values in  homepage_featured  :  2
Unique values in  num_orders  :  1250
Unique values in  city_code  :  51
Unique values in  region_code  :  8
Unique values in  center_type  :  3
Unique values in  op_area  :  30
Unique values in  category  :  14
Unique values in  cuisine  :  4


In [67]:
for col in test.columns:
    print("Unique values in ",col," : ",test[col].nunique())

Unique values in  id  :  32573
Unique values in  week  :  10
Unique values in  center_id  :  77
Unique values in  meal_id  :  51
Unique values in  checkout_price  :  1397
Unique values in  base_price  :  1179
Unique values in  emailer_for_promotion  :  2
Unique values in  homepage_featured  :  2
Unique values in  city_code  :  51
Unique values in  region_code  :  8
Unique values in  center_type  :  3
Unique values in  op_area  :  30
Unique values in  category  :  14
Unique values in  cuisine  :  4


In [68]:
train.describe()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,city_code,region_code,op_area
count,4.565480e+05,456548.000000,456548.000000,456548.000000,456548.000000,456548.000000,456548.000000,456548.00000,456548.000000,456548.000000,456548.000000,456548.000000
mean,1.250096e+06,74.768771,82.105796,2024.337458,332.238933,354.156627,0.081152,0.10920,261.872760,601.553399,56.614566,4.083590
std,1.443548e+05,41.524956,45.975046,547.420920,152.939723,160.715914,0.273069,0.31189,395.922798,66.195914,17.641306,1.091686
min,1.000000e+06,1.000000,10.000000,1062.000000,2.970000,55.350000,0.000000,0.00000,13.000000,456.000000,23.000000,0.900000
25%,1.124999e+06,39.000000,43.000000,1558.000000,228.950000,243.500000,0.000000,0.00000,54.000000,553.000000,34.000000,3.600000
50%,1.250184e+06,76.000000,76.000000,1993.000000,296.820000,310.460000,0.000000,0.00000,136.000000,596.000000,56.000000,4.000000
75%,1.375140e+06,111.000000,110.000000,2539.000000,445.230000,458.870000,0.000000,0.00000,324.000000,651.000000,77.000000,4.500000
max,1.499999e+06,145.000000,186.000000,2956.000000,866.270000,866.270000,1.000000,1.00000,24299.000000,713.000000,93.000000,7.000000


In [69]:
test.describe()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,city_code,region_code,op_area
count,3.257300e+04,32573.000000,32573.000000,32573.000000,32573.000000,32573.000000,32573.000000,32573.000000,32573.000000,32573.000000,32573.000000
mean,1.248476e+06,150.477819,81.901728,2032.067909,341.854440,356.493615,0.066435,0.081356,601.519971,56.712154,4.087726
std,1.441580e+05,2.864072,45.950455,547.199004,153.893886,155.150101,0.249045,0.273385,65.996677,17.641174,1.098835
min,1.000085e+06,146.000000,10.000000,1062.000000,67.900000,89.240000,0.000000,0.000000,456.000000,23.000000,0.900000
25%,1.123969e+06,148.000000,43.000000,1558.000000,214.430000,243.500000,0.000000,0.000000,556.000000,34.000000,3.600000
50%,1.247296e+06,150.000000,76.000000,1993.000000,320.130000,321.130000,0.000000,0.000000,596.000000,56.000000,4.000000
75%,1.372971e+06,153.000000,110.000000,2569.000000,446.230000,455.930000,0.000000,0.000000,651.000000,77.000000,4.500000
max,1.499996e+06,155.000000,186.000000,2956.000000,1113.620000,1112.620000,1.000000,1.000000,713.000000,93.000000,7.000000


In [70]:
train_df = train.drop(['id'], axis=1)
test_df = test.drop(['id'], axis=1)

In [71]:
num_vars = ['checkout_price','base_price','num_orders','week']

In [72]:
cat_vars = [col for col in test_df.columns if col not in num_vars]

In [73]:
def cat_enc_prob(df_train,df_test,variable):
    
    prob_df = df_train[variable].value_counts().to_dict()
    df_train[variable] = df_train[variable].map(prob_df)
    df_test[variable] = df_test[variable].map(prob_df)

In [74]:
for col in cat_vars:
    cat_enc_prob(train_df,test_df,col)

In [75]:
train_df.head()

,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,city_code,region_code,center_type,op_area,category,cuisine
0,1,5146,11092,136.83,152.29,419498,406693,177,11833,191228,99593,9512,127890,118216
1,2,5146,11092,135.83,152.29,419498,406693,323,11833,191228,99593,9512,127890,118216
2,3,5146,11092,132.92,133.92,419498,406693,96,11833,191228,99593,9512,127890,118216
3,4,5146,11092,135.86,134.86,419498,406693,163,11833,191228,99593,9512,127890,118216
4,5,5146,11092,146.50,147.50,419498,406693,215,11833,191228,99593,9512,127890,118216


In [76]:
test_df.head()

,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,city_code,region_code,center_type,op_area,category,cuisine
0,146,5146,11092,158.11,159.11,419498,406693,11833,191228,99593,9512,127890,118216
1,147,5146,11092,159.11,159.11,419498,406693,11833,191228,99593,9512,127890,118216
2,149,5146,11092,157.14,158.14,419498,406693,11833,191228,99593,9512,127890,118216
3,150,5146,11092,159.14,157.14,419498,406693,11833,191228,99593,9512,127890,118216
4,151,5146,11092,160.11,159.11,419498,406693,11833,191228,99593,9512,127890,118216


In [83]:
train_columns = train_df.columns.values
train_columns

array(['week', 'center_id', 'meal_id', 'checkout_price', 'base_price',
       'emailer_for_promotion', 'homepage_featured', 'num_orders',
       'city_code', 'region_code', 'center_type', 'op_area', 'category',
       'cuisine'], dtype=object)

In [84]:
test_columns = test_df.columns.values
test_columns

array(['week', 'center_id', 'meal_id', 'checkout_price', 'base_price',
       'emailer_for_promotion', 'homepage_featured', 'city_code',
       'region_code', 'center_type', 'op_area', 'category', 'cuisine'],
      dtype=object)

In [72]:
#train_df = pd.get_dummies(train_df, drop_first=True, columns=['emailer_for_promotion','homepage_featured'])
#test_df = pd.get_dummies(test, drop_first=True, columns=['emailer_for_promotion','homepage_featured'])

In [85]:
X = train_df.drop(['num_orders'], axis=1).values
y = train_df['num_orders'].values

In [100]:
test = test_df.values

In [87]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=101)

In [88]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)

In [89]:
meanX = sc_X.mean_
varX = sc_X.var_

In [90]:
meanX

array([7.47457437e+01, 6.02206772e+03, 9.67362036e+03, 3.32034379e+02,
       3.53954572e+02, 3.88379964e+05, 3.67721225e+05, 1.74554843e+04,
       1.31762681e+05, 1.92542965e+05, 2.58639842e+04, 5.58226272e+04,
       1.14637823e+05])

In [91]:
varX

array([1.72637122e+03, 4.92988428e+05, 3.80953089e+06, 2.33906450e+04,
       2.58130658e+04, 1.09326985e+10, 1.23878110e+10, 3.05077359e+08,
       3.34007330e+09, 6.72937313e+09, 2.59534934e+08, 2.05241987e+09,
       5.43077273e+07])

In [92]:
X_train

array([[-1.5823427 , -0.18524699,  0.01607727, ..., -0.82283748,
        -0.6208911 ,  1.12454236],
       [-0.33082715, -0.44160928, -0.21857769, ...,  0.88242943,
        -0.60179771,  1.12454236],
       [-0.81217929,  0.95983791, -2.55744203, ...,  1.40713172,
        -0.9328307 ,  0.48554675],
       ...,
       [-0.35489476, -0.18524699,  0.74719652, ..., -0.82283748,
        -0.49734694,  1.12454236],
       [-1.10099057, -0.05564161,  0.73182611, ..., -1.2340705 ,
        -0.49734694,  1.12454236],
       [ 0.00611934,  0.94701979, -0.70684444, ..., -0.01092385,
        -0.6208911 ,  1.12454236]])

In [93]:
X_test = sc_X.transform(X_test)
X_test

array([[-2.10489119e-01,  1.09569647e-01,  7.41560707e-01, ...,
        -9.11229305e-01,  1.59076338e+00,  1.12454236e+00],
       [-1.07692296e+00,  9.88322608e-01, -2.80131923e+00, ...,
        -8.22837485e-01, -9.52409704e-01,  4.85546748e-01],
       [-1.38286299e-01,  5.22597781e-01, -3.25577107e+00, ...,
        -8.33886462e-01, -1.00732803e+00, -1.60703158e+00],
       ...,
       [ 1.57051378e+00, -1.54254289e+00,  6.54974054e-01, ...,
        -2.24073419e-02, -5.00724153e-01, -1.60703158e+00],
       [ 1.35390532e+00,  1.27174536e+00, -1.27554968e+00, ...,
        -3.21412538e-01, -7.77169925e-01, -2.74897383e-01],
       [-1.58234270e+00, -4.28791163e-01,  1.21920238e-03, ...,
        -1.56298603e-01, -5.85573845e-01, -2.74897383e-01]])

In [102]:
test = sc_X.transform(test)
test

array([[ 1.71491942, -1.24772626,  0.72670264, ..., -1.01501521,
         1.59076338,  0.48554675],
       [ 1.73898703, -1.24772626,  0.72670264, ..., -1.01501521,
         1.59076338,  0.48554675],
       [ 1.78712224, -1.24772626,  0.72670264, ..., -1.01501521,
         1.59076338,  0.48554675],
       ...,
       [ 1.78712224, -0.23794368, -3.74864895, ..., -0.01092385,
        -1.00732803, -1.60703158],
       [ 1.81118985, -0.23794368, -3.74864895, ..., -0.01092385,
        -1.00732803, -1.60703158],
       [ 1.85932506, -0.23794368, -3.74864895, ..., -0.01092385,
        -1.00732803, -1.60703158]])

In [94]:
# Fitting Simple Linear Regression to the Training set
from sklearn.linear_model import LinearRegression
regressor = LinearRegression(n_jobs=-1)
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [95]:
b1 = regressor.coef_
b0 = regressor.intercept_

In [96]:
b1

array([  2.69532656,  92.39308297, 107.12434815, -70.91470738,
       -64.13331091, -81.13677972, -71.90516478,  25.37962456,
       -10.11021111, -16.41930221, -28.84993971, -58.5984442 ,
       -11.85212463])

In [97]:
b0

262.0939818448529

In [98]:
# Predicting the Test set results
y_pred = regressor.predict(X_test)
y_pred

array([ 363.2161862 ,  142.8284251 , -275.79183892, ...,   -1.69297501,
        178.63642025,  120.63034965])

In [99]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test,y_pred)
print('MSE: ',mse)
rmse = mse**0.5
print('RMSE: ',rmse)

MSE:  114630.73941395256
RMSE:  338.5716163737778


In [ ]:
lr_results = regressor.predict()
lr_results

In [ ]:
test = sc_X.transform(test)
test

In [103]:
lr_results = regressor.predict(test)
lr_results

array([ 268.7237837 ,  268.32497712,  269.75536045, ..., -399.96031169,
       -399.89544163, -401.02772913])

In [104]:
len(lr_results)

32573

In [105]:
test.shape

(32573, 13)

# prepare submission file

In [107]:
test_df['num_orders'] = lr_results

In [108]:
subFood = pd.read_csv("Sub.csv")

In [109]:
re = pd.concat([subFood['id'],test['num_orders']], axis=1)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [110]:
#re['num_orders'].value_counts()

In [111]:
re.head()

NameError: name 're' is not defined

In [112]:
re.to_csv("Food_lr.csv",index=False)

NameError: name 're' is not defined